<a href="https://colab.research.google.com/github/nirb28/ee-predict/blob/main/cd_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
def resolve_path_gdrive(relativePath):
    if os.path.exists('/content/drive'):
        return '/content/drive/MyDrive/work/gdrive-workspaces/git/ee-predict/' + relativePath
    else:
        return relativePath
from google.colab import drive
drive.mount('/content/drive', force_remount=False)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
from sklearn.neighbors import NearestNeighbors
import pandas as pd
import numpy as np, os
import warnings
warnings.filterwarnings('ignore')

In [3]:
#DATAFILE_NAME = "reduced_dim_space_ddG.csv"
DATASET = "LARGE" # "LARGE" or "REDUCED"
if DATASET == "REDUCED":
  DATAFILE_NAME = "reduced_dim_space_ddG.csv"
  X_COLS = ['x', 'y', 'z']
else:
  DATAFILE_NAME = "large_cat_desc_col_names.csv"
  X_COLS = ['3954', '3955', '3957', '3958', '3959', '3960', '3961', '3962', '3963',
       '3964', '3965', '3966', '3967', '3968', '3969', '3970', '3971', '3972',
       '3973']

df = pd.read_csv(resolve_path_gdrive('data/' + DATAFILE_NAME))
df = df[df['ddG'] != 0]
df.set_index('Catalyst', inplace=True)
## Our dataset has another issue: There are only a handful of unique ddG values.
df['ddG'].value_counts()

ddG
-0.672194    635
 1.463011    319
 0.751276    319
-1.146684    317
 0.276786    313
Name: count, dtype: int64

In [4]:
# method to make a regression model from the provided catalysts
from sklearn.cross_decomposition import PLSRegression
def make_pls_model(df_catalysts):
    X = df_catalysts[X_COLS]
    y = df_catalysts['ddG']
    model = PLSRegression(n_components=2)
    # fitting the model
    model.fit(X, y)
    return model

from joblib import dump, load
def load_model(path):
  return load(path)

pls_large_saved_model = load_model(resolve_path_gdrive('models/pls_large.joblib'))
print(pls_large_saved_model)

def pls_predict_ee(pls_model, properties):
    return pls_model.predict(properties.reshape(1,-1))

## Add a predicted ddG column to the df
df['pred_ddG'] = df.apply(lambda x: pls_predict_ee(pls_large_saved_model, np.array(x[X_COLS]))[0][0], axis=1)

PLSRegression(n_components=3)


In [5]:
# start with a few randomly selected catalyst
START_SAMPLE_SIZE = 3
random_catalyst_df = df.sample(START_SAMPLE_SIZE)
random_catalyst_df = df.loc[['5_5_4', '16_11_1', '5_5_1']]
random_catalyst_df

,1,2,3,4,5,6,7,8,9,10,...,3966,3967,3968,3969,3970,3971,3972,3973,ddG,pred_ddG
Catalyst,,,,,,,,,,,,,,,,,,,,,
5_5_4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.643031,-0.505083,-0.825677,-0.297732,-0.236059,-0.267225,0.028798,-0.691898,-0.672194,-0.788961
16_11_1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.605952,-0.471700,-0.768480,-0.178536,-0.298590,-0.238640,-0.277510,-0.230633,-0.672194,-0.639691
5_5_1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.605952,-0.471700,-0.768480,-0.178536,-0.298590,-0.238640,-0.277510,-0.230633,-0.672194,-0.639691


In [6]:
original_ee = df['ddG']
# Function to optimize catalyst properties using coordinate descent
def optimize_catalysts(catalysts, pls_model=None, iterations=100, cd_iterations=10, step_size=0.001):
    if pls_model == None:
        print("Creating a new model. # data items: " + str(len(catalysts)))
        pls_model = make_pls_model(random_catalyst_df)
    optimized_catalysts = np.copy(catalysts)
    for _ in range(iterations):
        for i in range(len(optimized_catalysts)):
            ## we have few options for the original_ee. We can use from the dataset or get the prediction
            ## from the model. The prediction makes sense as we want to traverse in the direction where
            ## ee improves according to the model

            ## when using ddG from dataset
            #original_ee = df.iloc[df.index.get_loc(catalysts[i:i+1].index[0])]['ddG']
            ## when using the model
            original_ee = pls_predict_ee(pls_model, optimized_catalysts[i])
            for x in range(len(optimized_catalysts[i])):
                for cd in range(cd_iterations):
                    old_value = optimized_catalysts[i, x]
                    optimized_catalysts[i, x] = old_value + step_size
                    new_ee = pls_predict_ee(pls_model, optimized_catalysts[i])
                    if new_ee < original_ee:
                        optimized_catalysts[i, x] = old_value - step_size
                        new_ee = pls_predict_ee(pls_model, optimized_catalysts[i])
                    if new_ee < original_ee:
                        optimized_catalysts[i, x] = old_value
                        break
                    #print('Found a direction for higher ddG')
    return optimized_catalysts

new_optimized_catalysts = optimize_catalysts(random_catalyst_df[X_COLS], pls_large_saved_model)

In [7]:
random_catalyst_df[X_COLS]

,3954,3955,3957,3958,3959,3960,3961,3962,3963,3964,3965,3966,3967,3968,3969,3970,3971,3972,3973
Catalyst,,,,,,,,,,,,,,,,,,,
5_5_4,0.758129,0.677277,0.686929,0.574038,-0.399013,-0.833174,-0.693664,-0.764237,-0.187753,-0.896976,-0.938972,-0.643031,-0.505083,-0.825677,-0.297732,-0.236059,-0.267225,0.028798,-0.691898
16_11_1,1.192046,0.575046,1.190427,1.005503,-0.429561,-0.819943,-1.060149,-0.944026,-0.177977,-0.893660,-0.833714,-0.605952,-0.471700,-0.768480,-0.178536,-0.298590,-0.238640,-0.277510,-0.230633
5_5_1,1.192046,0.575046,1.190427,1.005503,-0.429561,-0.819943,-1.060149,-0.944026,-0.177977,-0.893660,-0.833714,-0.605952,-0.471700,-0.768480,-0.178536,-0.298590,-0.238640,-0.277510,-0.230633


In [8]:
new_optimized_catalysts

array([[ 1.75812941,  1.67727654,  1.68692902,  1.57403813,  0.60098717,
         0.16682611,  0.30633566,  0.23576264,  0.81224705,  0.10302372,
         0.06102794,  0.35696864,  0.49491701,  0.17432342,  0.70226783,
         0.76394068,  0.73277519,  1.02879822,  0.30810159],
       [ 2.19204621,  1.57504612,  2.19042725,  2.00550337,  0.57043889,
         0.1800573 , -0.0601493 ,  0.05597431,  0.82202323,  0.10633971,
         0.16628637,  0.39404832,  0.52829963,  0.23152003,  0.82146371,
         0.70141005,  0.76136021,  0.72248989,  0.7693672 ],
       [ 2.19204621,  1.57504612,  2.19042725,  2.00550337,  0.57043889,
         0.1800573 , -0.0601493 ,  0.05597431,  0.82202323,  0.10633971,
         0.16628637,  0.39404832,  0.52829963,  0.23152003,  0.82146371,
         0.70141005,  0.76136021,  0.72248989,  0.7693672 ]])

In [9]:
# Perform k-nearest neighbors analysis
num_neighbors = 1  # Number of neighbors to consider
knn = NearestNeighbors(n_neighbors=num_neighbors)
knn.fit(df[X_COLS])  # Using all the catalysts

NearestNeighbors(n_neighbors=1)

In [10]:
## Recursive Loop
def find_unique_neighbors_add(knn, point, addToDf, neighbor_number=1):
    distances, indices = knn.kneighbors([point], neighbor_number)
    neighbor_catalyst = df.iloc[[indices[0][neighbor_number-1]]]
    if neighbor_catalyst.index[0] in addToDf.index:
        #get next neighbor
        neighbor_catalyst = find_unique_neighbors_add(knn, point, addToDf, neighbor_number+1)
    return neighbor_catalyst

def process_neighbors(knn, for_points, addToDf):
    for i in range(len(for_points)):
        neighbor_catalyst = find_unique_neighbors_add(knn, for_points[i], addToDf)
        if len(neighbor_catalyst) > 0:
            addToDf = pd.concat([addToDf, neighbor_catalyst], ignore_index=False, sort=True)
    return addToDf

random_catalyst_df = process_neighbors(knn, new_optimized_catalysts, random_catalyst_df)
random_catalyst_df[X_COLS]

,3954,3955,3957,3958,3959,3960,3961,3962,3963,3964,3965,3966,3967,3968,3969,3970,3971,3972,3973
Catalyst,,,,,,,,,,,,,,,,,,,
5_5_4,0.758129,0.677277,0.686929,0.574038,-0.399013,-0.833174,-0.693664,-0.764237,-0.187753,-0.896976,-0.938972,-0.643031,-0.505083,-0.825677,-0.297732,-0.236059,-0.267225,0.028798,-0.691898
16_11_1,1.192046,0.575046,1.190427,1.005503,-0.429561,-0.819943,-1.060149,-0.944026,-0.177977,-0.893660,-0.833714,-0.605952,-0.471700,-0.768480,-0.178536,-0.298590,-0.238640,-0.277510,-0.230633
5_5_1,1.192046,0.575046,1.190427,1.005503,-0.429561,-0.819943,-1.060149,-0.944026,-0.177977,-0.893660,-0.833714,-0.605952,-0.471700,-0.768480,-0.178536,-0.298590,-0.238640,-0.277510,-0.230633
1_1_1,1.192046,0.575046,1.190427,1.005503,-0.429561,-0.819943,-1.060149,-0.944026,-0.177977,-0.893660,-0.833714,-0.605952,-0.471700,-0.768480,-0.178536,-0.298590,-0.238640,-0.277510,-0.230633
1_10_1,1.192046,0.575046,1.190427,1.005503,-0.429561,-0.819943,-1.060149,-0.944026,-0.177977,-0.893660,-0.833714,-0.605952,-0.471700,-0.768480,-0.178536,-0.298590,-0.238640,-0.277510,-0.230633
1_2_1,1.192046,0.575046,1.190427,1.005503,-0.429561,-0.819943,-1.060149,-0.944026,-0.177977,-0.893660,-0.833714,-0.605952,-0.471700,-0.768480,-0.178536,-0.298590,-0.238640,-0.277510,-0.230633


In [11]:
# sample prediction with model
properties1 = (np.array([ 0.75813041,  0.67727654,  0.68692902,  0.57403813, -0.39901283,
       -0.83317389, -0.69366434, -0.76423736, -0.18775295, -0.89697628,
       -0.93897206, -0.64303136, -0.50508299, -0.82567658, -0.29773217,
       -0.23605932, -0.26722481,  0.02879822, -0.69189841]))
pls_predict_ee(pls_large_saved_model, properties1)
# random_catalyst_df = random_catalyst_df[~random_catalyst_df.index.duplicated(keep='first')]

array([[-0.78896142]])

In [14]:
# We will now do all this in a single loop
highest_ddg_index = df.index[df['ddG'] == df['ddG'].max()]
print("The max value (target) of the ddG is: " + str(df['ddG'].max()))

not_highest_index=[i for i in df.index if i not in highest_ddg_index]
random_catalyst_df = df.loc[not_highest_index].sample(START_SAMPLE_SIZE)
while(len(random_catalyst_df) < len(df)):
    print("The current maximum ddg is at: " + str(random_catalyst_df['pred_ddG'].max()))
    new_optimized_catalysts = optimize_catalysts(random_catalyst_df[X_COLS], pls_large_saved_model)
    random_catalyst_df = process_neighbors(knn, new_optimized_catalysts, random_catalyst_df)
    if len(pd.concat([highest_ddg_index.to_series(), random_catalyst_df.index.to_series()], axis=1, join='inner')) > 0:
        print("Found the highest ddG. Exiting.")
        break

The max value (target) of the ddG is: 1.46301068
The current maximum ddg is at: 0.33990871537413136
The current maximum ddg is at: 0.33990871537413136
The current maximum ddg is at: 0.33990871537413136
The current maximum ddg is at: 0.33990871537413136


KeyboardInterrupt: 

In [13]:
random_catalyst_df['pred_ddG'].max()

1.3886042742878204

In [82]:
highest_ddg_index

Index(['1_1_5', '1_10_5', '1_11_5', '1_2_5', '1_3_5', '1_4_5', '1_5_5',
       '1_6_5', '1_7_5', '1_8_5',
       ...
       '9_10_5', '9_11_5', '9_2_5', '9_3_5', '9_4_5', '9_5_5', '9_6_5',
       '9_7_5', '9_8_5', '9_9_5'],
      dtype='object', name='Catalyst', length=319)

In [83]:
len(pd.concat([highest_ddg_index.to_series(), random_catalyst_df.index.to_series()], axis=1, join='inner'))

0

In [ ]:
not_highest_index=[i for i in df.index if i not in highest_ddg_index]
df.loc[not_highest_index].index

Index(['1_1_1', '1_1_2', '1_1_3', '1_1_4', '1_1_6', '1_10_1', '1_10_2',
       '1_10_3', '1_10_4', '1_10_6',
       ...
       '9_8_1', '9_8_2', '9_8_3', '9_8_4', '9_8_6', '9_9_1', '9_9_2', '9_9_3',
       '9_9_4', '9_9_6'],
      dtype='object', name='Catalyst', length=1584)